# eTranslation Web automation
 > GOAL: Automatically connect to eTranlation Web page and obtain the translation of legislative documents.
 
 > URL: https://webgate.ec.europa.eu/etranslation
 
 > AUTHOR: Edmundo

## Installation steps for Web automation - required browsers, libraries and/or drivers

In [1]:
# Install Chrome
#!apt-get update
#!apt-get install -y libasound2 libasound2-data libappindicator1 fonts-liberation
#!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
#!dpkg -i google-chrome*.deb

# Download Chrome driver for selenium
#!apt-get install -y libnss3
#!wget -P selenium-drivers/ https://chromedriver.storage.googleapis.com/76.0.3809.126/chromedriver_linux64.zip
#!unzip -o selenium-drivers/chromedriver_linux64.zip -d selenium-drivers/
#!rm selenium-drivers/chromedriver_linux64.zip
#!chmod +x selenium-drivers/chromedriver
#!ls -la selenium-drivers/

# Install Firefox
#!apt-get install -y firefox
#!apt-get install -y xvfb

#!wget -P selenium-drivers/ https://github.com/mozilla/geckodriver/releases/download/v0.24.0/geckodriver-v0.24.0-linux64.tar.gz
#!tar -xvzf selenium-drivers/geckodriver-v0.24.0-linux64.tar.gz -C selenium-drivers/
#!rm selenium-drivers/geckodriver-v0.24.0-linux64.tar.gz

## Initialization

In [2]:
from pyvirtualdisplay import Display
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import getpass
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.dpi'] = 350

def loginAndOpenTranslationTextPage(driver, user, password):
    driver.get("https://webgate.ec.europa.eu/etranslation")
    assert driver.title == "EU Login"
    driver.find_element(By.ID, "username").send_keys(user)
    driver.find_element(By.NAME, "whoamiSubmit").click()
    assert driver.title == "Login"
    driver.find_element(By.ID, "password").send_keys(password)
    driver.find_element(By.NAME, "_submit").click()
    driver.find_element(By.LINK_TEXT, "Translate text").click()
    assert driver.title == "Translate text snippet"

def translateToEnglish(driver, fromText):
    driver.execute_script("document.getElementById('translatedText').value = ''")
    driver.find_element(By.ID, "limitedtextarea").send_keys(fromText)
    driver.execute_script("$('#srcLangSelect').val('PT').trigger('change')")
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//select[@name='targetLanguages']/option[. = 'EN']")))
    driver.execute_script("$('#targetL').val('EN').trigger('change')")
    driver.find_element(By.ID, "submit").click()
    WebDriverWait(driver, 20).until(lambda driver: driver.execute_script("return document.getElementById('translatedText').value").strip() != '')
    assert driver.title == "Translate text snippet"
    #plt.imshow(Image.open(BytesIO(driver.get_screenshot_as_png())))
    return driver.execute_script("return document.getElementById('translatedText').value")


## Proxy settings

In [3]:
#import os
#os.environ['http_proxy'] = "http://HOST:PORT"
#os.environ['https_proxy'] = "http://HOST:PORT"

## User credentials

In [4]:
user = "YOUR_EMAIL"
print("Type your eTranslation password:")
password = getpass.getpass()

Type your eTranslation password:


## Input sample

In [5]:
documents = {
    'id': [101, 102],
    'text': ['Constituição Federal', 'Resolução do Congresso Nacional']
}
df = pd.DataFrame(documents)
df

,id,text
0,101,Constituição Federal
1,102,Resolução do Congresso Nacional


## Automatic translations

In [8]:
with Display(visible=0, size=(1024, 768)) as display:
    with webdriver.Chrome(executable_path="selenium-drivers/chromedriver") as driver:
        driver.implicitly_wait(10)
        loginAndOpenTranslationTextPage(driver, user, password)
        for index_label, row_series in df.iterrows():
            text = df.at[index_label, "text"]
            df.at[index_label, "en_text"] = translateToEnglish(driver, text)
df

,id,text,en_text
0,101,Constituição Federal,﻿Federal Constitution
1,102,Resolução do Congresso Nacional,﻿Resolution of the Federal Congress
